# Price Discovery Project Using Agents

This is a notebook that contains the experimentation of prioce discovery using agents.

## Import Libraries

In [2]:
import os
import json
from typing import Any, Union


import PIL
from dotenv import load_dotenv


from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain.agents.agent import AgentAction
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.messages import HumanMessage
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain_core.output_parsers import JsonOutputParser



## Specify Variables

In [3]:
# Specify variables
model_name = "ViT-B-16"
checkpoint = "openai"
embedding_model= OpenCLIPEmbeddings(model_name=model_name, checkpoint=checkpoint)
# db_dir = "db"

# Load environment variables
load_dotenv()

True

## Create Vector DB to store data

**In the case of creating a new db**

In [3]:
# Instantiate DB
# text_db = Chroma(persist_directory=db_dir, embedding_function=embedding_model, collection_name="text-collection")
# images_db = Chroma(persist_directory=db_dir, embedding_function=embedding_model, collection_name="images-collection")

In [4]:
# Add data to db 

#text_db.add_documents(text_data)
#images_db.add_images(image_data)

**If DB is already initialized**

In [4]:
# Instantiate DB
text_db = Chroma(persist_directory="../data/databases/text_db", embedding_function=embedding_model, collection_name="Text_Store")
# images_test_db = Chroma(persist_directory="Databases/image_db", embedding_function=embedding_model, collection_name="Image_Store")

## Create agent for retrieval

**Create Retriever from database**

In [5]:
# Instantiate retrievers from db

#text_retriever = text_db.as_retriever()
text_retriever = text_db.as_retriever()
#images_test_retriever = images_test_db.as_retriever()

In [6]:
text_retriever.get_relevant_documents("heels for women")

[]

**Instatiate the llms**

In [55]:
# Mixtral

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

mixtral = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.3
)



Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


Token is valid (permission: read).
Your token has been saved to C:\Users\Spectra\.cache\huggingface\token
Login successful


In [56]:

# Setup google api
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

gemini = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0.3)
gemini_vision = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY, temperature=0.1)

**Instantiate prompts**

In [63]:

# Create prompts
test_prompt = """
    
As an experienced product analyst adept at accurately determining product price ranges,utilize the available tools {tools} to answer the question asked. 
Return ONLY a reasonable, accurate and concise price range for my product after analysing its current pricing
and that of a similar products. 

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found use your own knowledge
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""

test_prompt_one = """    
As an experienced product analyst adept at accurately determining product price ranges, utilize ALL the available tools {tools} to answer the question asked accurately. 
Check whether the product is in the database, then search the internet and compare the prices.
If a tool encounters an error, use another tool.
Return ONLY a reasonable and accurate  PRICE RANGE for the product.
Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the analysis of your findings and observations.
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

test_prompt_two = """

As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-110 with the price of the product being $106

Use the following format:
Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of[{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the analysis of your findings and observations.
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer.

An example to follow is:

Question: what is the estimated, accurate and compact price range of a Sony WH-CH520 Wireless Headphones Bluetooth On-Ear Headset with Microphone, Black New?
Thought 1: I need to search for the Sony WH-CH520 Wireless Headphones to identify their price range.
Action 1: search text db
Action Input 1: : Sony WH-CH520 Wireless Headphones.
Observation 1: The search results providing various listings for products similar to the Sony WH-CH520 Wireless Headphones with different prices.
Action 2: search internet 
Action Input 2: Sony WH-CH520 Wireless Headphones Bluetooth On-Ear Headset with Microphone.
Observation 2: The search results provide various listings for the Sony WH-CH520 Wireless Headphones with different prices.
Thought 3: I need to check the obtained products for similarity.
Action 3: The products with great similarity to the Sony WH-CH520 Wireless Headphones are chosen.
Thought 4: I need to gather information on the range of prices offered for these headphones.
Action 4: Compare and analyze the search results to determine the price range.
Observation 4: Prices for the Sony WH-CH520 Wireless Headphones range from $80 to $100 across different retailers and platforms, based on comparing to very similar products with similar specifications and brand.
Thought 5: The price range of the Sony WH-CH520 Wireless Headphones is between $80 and $100.
Final Answer: $80 - $100


Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

test_prompt_three = """
As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-$110 with the price of the product being $106. It SHOULD be in the JSON format with the key being "price_range": 
Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer. 
Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

test_prompt_four = """

Given the product description and the product image,  utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. 
Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-110 with the price of the product being $106


Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer.

Begin!

Question: {input} 
Image: {image}
Thought: {agent_scratchpad}

"""

test_prompt_five = """
As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases, to precisely answer the given question.
Begin by checking the database, then proceed to search the internet. 
Then pick the most similar products and use them to come up with an accurate price range. 
Similar products are those with closely matching specifications based on product information and brand.
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product. If a tool encounters an error, use another tool.

Return ONLY the ESTIMATE of the PRICE RANGE for the product. The price range should be in the JSON format with the key being "price_range" and the value being a price range.  Example: "price_range": "$100-$110"

Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer. {{"price_range": "<price_range>}}


Begin!

Question: {input} 
Thought: {agent_scratchpad}
"""

In [64]:
prompt_template = PromptTemplate.from_template(test_prompt_three)

**Instantiate Agents**

In [50]:
# Instantiate error handler
class RetryCallbackHandler(BaseCallbackHandler):
    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        # Log the error or perform any necessary actions
        
        # Retry logic
        max_retries = 1
        current_retry = kwargs.get('retry_count', 0)
        if current_retry < max_retries:
            print(f"Retrying tool execution (Attempt {current_retry + 1})")
            # Increment retry count
            kwargs['retry_count'] = current_retry + 1
            # Re-run the tool
            return AgentAction.RERUN
        else:
            print("Maximum retries reached. Switching to another tool.")
            return AgentAction.CHANGE_TOOL

In [39]:
# The error handler
error_handler = RetryCallbackHandler()

In [65]:
# Create tools
search = SerpAPIWrapper()

tool_search_item = Tool(
    name="search internet",
    description="Searches the internet for the price of a product by using its description.",
    func=search.run,
    handle_tool_error=True
)

# Tool for searching product price using image URL
tool_search_image = Tool(
    name="search image",
    description="Searches for the price of a product using the image.",
    func=search.run,
    handle_tool_error=True
)

# Tool for retrieving product information from the text database based on the product description
tool_retrieve_from_text_db = create_retriever_tool(
    text_retriever,
    'search text db',
    'Query a retriever for product price using its product description.')

# List of all tools
tools = [tool_search_item, tool_search_image, tool_retrieve_from_text_db]

In [66]:

# Instantiate agents
parser = JsonOutputParser()

serp_agent = create_react_agent(mixtral, tools, prompt_template) 



serp_agent_executor = AgentExecutor(agent=serp_agent,
                                    tools=tools,
                                    verbose=True,
                                    return_intermediate_steps=True,
                                    handle_parsing_errors=True,
                                    callbacks=[error_handler])


In [67]:
description = "Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life"
image = PIL.Image.open("../data/test_data/51RE0DBn5VL._AC_SL1500_.jpg")

**Run the pipeline**

In [68]:
vision_chat_template = HumanMessage(
        content=[
            {
        "type": "text",
        "text": f"""Your task is to generate a searchable prompt about the product in the image that I can feed to Google to find this exact product. You should:

1. Given the description {description}, extract relevant details about the product from the image, such as brand, color, and your thoughts on quality.

2. Use these details to generate a concise searchable prompt that would allow me to find this exact product on Google.

3. Return ONLY the searchable prompt in the following JSON format: {{"search_prompt": "<YOUR_SEARCHABLE_PROMPT>}}. Do not include any other information or text.

Please strictly follow these instructions and provide your response in the specified JSON format.""",
    },  
    {"type": "image_url", "image_url": image}
                ])

vision_chat_template_one = HumanMessage(
        content=[
            {
        "type": "text",
        "text": f"""Your task is to generate a searchable prompt about the product in the image that I can feed to Google to find this exact product. You should:

1. Given the description {description}, extract relevant details about the product from the image, such as brand, color, and your thoughts on quality.

2. Use these details and the given description to generate a concise and well structured searchable prompt that would allow me to find this exact product on Google.

3. Return ONLY the searchable prompt in the following JSON format: {{"search_prompt": "<YOUR_SEARCHABLE_PROMPT>}}. Do not include any other information or text.

Please strictly follow these instructions and provide your response in the specified JSON format.""",
    },  
    {"type": "image_url", "image_url": image}
                ])


# define the parser object
parser = JsonOutputParser()

# create a chain 
chain =  gemini_vision | parser

response = chain.invoke([vision_chat_template_one])

search_prompt = response["search_prompt"]

result = serp_agent_executor.invoke({"input": f"what is the estimated, accurate and compact price range of a/an {search_prompt}. ALWAYS return a price range and not a price"})
output_str = result["output"]
try:
    price_dict = json.loads(output_str)
    print(price_dict["price_range"])
    
except:
    end_index = output_str.find('}') + 1

    # Extract the substring containing the dictionary
    dict_string = output_str[:end_index]

    # Parse the dictionary string using json.loads()
    price_dict = json.loads(dict_string)
    print(price_dict["price_range"])



> Entering new AgentExecutor chain...
I should first check the database for the price of the Bose Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, Stone White.

Action: search text db
Action Input: Bose Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, Stone White

ObservationAction: search internet
Action Input: Bose Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, Stone White

Observation[{'position': 1, 'block_position': 'top', 'title': 'Bose - Wireless - Noise Canceling - Over-ear - QuietComfort Ultra Headphones ,White', 'price': '$429.00', 'extracted_price': 429.0, 'link': 'https://www.qvc.com/qvc.product.E319510.html?colorId=202&sizeId=000&ref=GBA', 'source': 'QVC', 'thumbnail': 'https://serpapi.com/searches/65f1c85a0ead41801d9eee13/images/d25

In [80]:
image_bytes

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x04\xa4\x03\xc0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\